In [13]:
import json, pprint, requests, textwrap, time
host = 'http://150.140.142.92:8998'

In [ ]:
#get number of sessions
headers = {'Content-Type': 'application/json'}
r = requests.get(host+'/sessions', headers=headers)
con_var=r.json()['total']
print ("Waiting... active sesssions: ", con_var)
while con_var!=0:
    r = requests.get(host+'/sessions', headers=headers)
    con_var=r.json()['total']
    print ("Waiting... active sesssions: ", con_var)
    time.sleep(60)
print ("Good to go!")

Waiting... active sesssions:  1
Waiting... active sesssions:  1


In [ ]:
#create new session - DON'T DO THIS IF THERE ARE OTHER ACTIVE SESSIONS!
data = {'kind': 'pyspark',\
        'name':'bbjob' #,
        'conf':{
            'master':'spark://localhost:7077'}\
       }#, 'heartbeatTimeoutInSecond':240}

r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

In [51]:
#get session information
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()


{'id': 18,
 'name': 'bbjob',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ',
  '\nstderr: ',
  '21/06/02 17:32:45 WARN Utils: Your hostname, administrator-OptiPlex-390 resolves to a loopback address: 127.0.1.1; using 150.140.142.92 instead (on interface enp4s0)',
  '21/06/02 17:32:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address',
  '21/06/02 17:32:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
  'log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).',
  'log4j:WARN Please initialize the log4j system properly.',
  'log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.']}

In [47]:
#show session state, repeat until idle to proceed with code submission
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is idle


In [48]:
#send some python statements for execution
codefile = open('test.py', 'r')
code = codefile.read()
codefile.close()
data = {
    'kind': 'pyspark',
    'code': textwrap.dedent(code)
}

statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)

In [19]:
#show session state, repeat until idle to proceed with result extraction
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is idle


In [20]:
#get stats from the latest statement in the session
statements_url = session_url + '/statements'
print(statements_url)
r = requests.get(statements_url, headers=headers)
slist = r.json()
last_statement = slist['statements'][slist['total_statements']-1]
if last_statement['progress']==1:
    print ("Job completed")

print("Total Time %s ms" % (last_statement['completed'] - last_statement['started']))

http://150.140.142.92:8998/sessions/5/statements
Job completed
Total Time 6231 ms


In [21]:
#get last statement output
output = last_statement['output']
if output['status']=="error":
    pprint.pprint(output['traceback'])
else:
    fres = output['data']['text/plain'].replace("'","")
    #pprint.pprint(output['data']['text/plain'])
    fres = json.loads(fres)
print(fres[0]['author'])

Lisa Gardner


In [22]:
#close the session
r = requests.delete(session_url, headers=headers)
pprint.pprint(r.json())

{'msg': 'deleted'}


In [23]:
#close an arbitrary session
sessionid = 54
r = requests.delete("http://150.140.142.92:8998/sessions/"+str(sessionid), headers={'Content-Type': 'application/json'})
pprint.pprint(r.json())

{'msg': 'deleted'}
